In [1]:
from pymatgen.analysis.interfaces.substrate_analyzer import SubstrateAnalyzer
from pymatgen.analysis.interfaces.coherent_interfaces import CoherentInterfaceBuilder
from pymatgen.analysis.interfaces.zsl import ZSLGenerator
from pymatgen.core.structure import Structure
import crystal_toolkit

In [14]:
substrate = Structure.from_file("LiCoO2.cif")
substrate.add_oxidation_state_by_element({"Co": 3, "Li": 1, "O": -2})
# film = Structure.from_file("Li.cif")
# film.add_oxidation_state_by_element({"Li": 1})

# film = Structure.from_file("Li3PO4.cif")
# film.add_oxidation_state_by_element({"Li": 1, 'O':-2, 'P':5})

film = Structure.from_file("Li2SO4.cif")
film.add_oxidation_state_by_element({"Li": 1, 'O':-2, 'S':4})

# film = Structure.from_file("Li2ZnCl4.cif")
# film.add_oxidation_state_by_element({"Li": 1, 'Cl':-1, 'Zn':2})
film
# substrate

If you see this text, the Crystal Toolkit Jupyter Lab 

extension is not installed. You can install it by running 

"pip install crystaltoolkit-extension" 

from the same environment you run "jupyter lab". 

This only works in Jupyter Lab 3.x or above.


Structure Summary
Lattice
    abc : 8.38784496 4.92078882 8.20029857
 angles : 90.0 107.80461619 90.0
 volume : 322.25489542769844
      A : 7.986107116583605 0.0 -2.5647682537512275
      B : 7.913231955717569e-16 4.92078882 3.0131141388465407e-16
      C : 0.0 0.0 8.20029857
    pbc : True True True
PeriodicSite: Li+ (1.0296, 2.8280, 3.2653) [0.1289, 0.5747, 0.4385]
PeriodicSite: Li+ (6.9565, 2.0928, 2.3702) [0.8711, 0.4253, 0.5615]
PeriodicSite: Li+ (6.9565, 0.3676, -1.7299) [0.8711, 0.0747, 0.0615]
PeriodicSite: Li+ (1.0296, 4.5532, 7.3655) [0.1289, 0.9253, 0.9385]
PeriodicSite: Li+ (3.0246, 2.8059, 0.6551) [0.3787, 0.5702, 0.1983]
PeriodicSite: Li+ (4.9615, 2.1149, 4.9804) [0.6213, 0.4298, 0.8017]
PeriodicSite: Li+ (4.9615, 0.3455

In [15]:
substrate_millers=[(1,0,4)]

zsl_params = {
        'max_area_ratio_tol':0.09,
        'max_area':400,
        'max_length_tol':0.09,
        'max_angle_tol':0.05,
        'bidirectional':False,
}


'''
def __init__(self, film_max_miller=1, substrate_max_miller=1, **kwargs):
        """Initialize the substrate analyzer.

        Args:
            zslgen (ZSLGenerator): Defaults to a ZSLGenerator with standard
                tolerances, but can be fed one with custom tolerances
            film_max_miller (int): maximum miller index to generate for film
                surfaces
            substrate_max_miller (int): maximum miller index to generate for
                substrate surfaces.
'''

sub_analyzer = SubstrateAnalyzer(film_max_miller =4)
sub_analyzer.calculate(film=film,substrate=substrate)
matches = list(sub_analyzer.calculate(film=film,substrate=substrate, substrate_millers=substrate_millers))
len(matches)
# dir(matches[0])

film_millers = []
for match in matches:
    print(match.match_area,match.von_mises_strain, match.film_miller)
    hkl = match.film_miller
    if hkl not in film_millers:
        film_millers.append(hkl)
        
# output: surface close-pack param 

288.9236959214295 0.011612376973971779 (0, 0, 1)
330.19850962449084 0.011326894024802008 (0, 0, 1)
330.19850962449084 0.011326894024801977 (0, 0, 1)
330.19850962449084 0.008232321144669853 (0, 0, 1)
282.4635626674259 0.02525251549475447 (1, 0, 0)
137.63447429013445 0.017853812508152798 (3, 1, 0)
137.63447429013445 0.017853812508152572 (3, 1, 0)


In [ ]:
 """
    CoherentInterfaceBuilder
    
    Args:
        substrate_structure (Structure): substrate structure
        film_structure (Structure): film structure
        film_miller (tuple[int, int, int]): miller index for the film layer
        substrate_miller (tuple[int, int, int]): miller index for the substrate layer
        zslgen (ZSLGenerator | None): BiDirectionalZSL if you want custom lattice matching tolerances for coherency.
        termination_ftol (float): tolerance to distinguish different terminating atomic planes.
        label_index (bool): If True add an extra index at the beginning of the termination label.
        filter_out_sym_slabs (bool): If True filter out identical slabs with different terminations.
            This might need to be set as False to find more non-identical terminations because slab
            identity separately does not mean combinational identity.
    """

In [16]:
substrate_miller = (1,0,4)
film_millers = [[0,0,1]]
i = -1
all_interfaces = []


'''
ZSLGenerator(
        max_area_ratio_tol=0.09,
        max_area=400,
        max_length_tol=0.03,
        max_angle_tol=0.01,
        bidirectional=False,
    ):
        """
        Initialize a Zur Super Lattice Generator for a specific film and
            substrate.

        Args:
            max_area_ratio_tol(float): Max tolerance on ratio of
                super-lattices to consider equal
            max_area(float): max super lattice area to generate in search
            max_length_tol: maximum length tolerance in checking if two
                vectors are of nearly the same length
            max_angle_tol: maximum angle tolerance in checking of two sets
                of vectors have nearly the same angle between them.
        """
'''

for film_miller in film_millers:
    zsl = ZSLGenerator(max_area=400,
                      max_area_ratio_tol=0.5,
        max_length_tol=1,
        max_angle_tol=0.01,
        bidirectional=False,)
    cib = CoherentInterfaceBuilder(film_structure=film,
                               substrate_structure=substrate,
                               film_miller= film_miller,
                               substrate_miller=substrate_miller,
                               zslgen=zsl
                              )
    
    terminations = cib.terminations
    
    for termination in terminations:
        interfaces=list(cib.get_interfaces(termination= termination, gap=  2.0, vacuum_over_film =  15.0, 
                                   film_thickness =  5, substrate_thickness = 5, in_layers =  False))
        len(interfaces)
        for interface in interfaces:
            # if interface.num_sites <200:
                i+=1
                all_interfaces.append(interface)
                print(i,'hkl_host: ',substrate_miller, 'hkl_film: ',film_miller, f'Natom: {interface.num_sites}', termination)
            

    

0 hkl_host:  (1, 0, 4) hkl_film:  [0, 0, 1] Natom: 92 ('Li2O2_P1_2', 'LiCoO2_P2/m_4')
1 hkl_host:  (1, 0, 4) hkl_film:  [0, 0, 1] Natom: 92 ('Li2O2_P1_2', 'LiCoO2_P2/m_4')
2 hkl_host:  (1, 0, 4) hkl_film:  [0, 0, 1] Natom: 128 ('Li2O2_P1_2', 'LiCoO2_P2/m_4')
3 hkl_host:  (1, 0, 4) hkl_film:  [0, 0, 1] Natom: 128 ('Li2O2_P1_2', 'LiCoO2_P2/m_4')
4 hkl_host:  (1, 0, 4) hkl_film:  [0, 0, 1] Natom: 156 ('Li2O2_P1_2', 'LiCoO2_P2/m_4')
5 hkl_host:  (1, 0, 4) hkl_film:  [0, 0, 1] Natom: 156 ('Li2O2_P1_2', 'LiCoO2_P2/m_4')
6 hkl_host:  (1, 0, 4) hkl_film:  [0, 0, 1] Natom: 156 ('Li2O2_P1_2', 'LiCoO2_P2/m_4')
7 hkl_host:  (1, 0, 4) hkl_film:  [0, 0, 1] Natom: 156 ('Li2O2_P1_2', 'LiCoO2_P2/m_4')
8 hkl_host:  (1, 0, 4) hkl_film:  [0, 0, 1] Natom: 156 ('Li2O2_P1_2', 'LiCoO2_P2/m_4')
9 hkl_host:  (1, 0, 4) hkl_film:  [0, 0, 1] Natom: 164 ('Li2O2_P1_2', 'LiCoO2_P2/m_4')
10 hkl_host:  (1, 0, 4) hkl_film:  [0, 0, 1] Natom: 164 ('Li2O2_P1_2', 'LiCoO2_P2/m_4')
11 hkl_host:  (1, 0, 4) hkl_film:  [0, 0, 1]

In [18]:
all_interfaces[2]


If you see this text, the Crystal Toolkit Jupyter Lab 

extension is not installed. You can install it by running 

"pip install crystaltoolkit-extension" 

from the same environment you run "jupyter lab". 

This only works in Jupyter Lab 3.x or above.


Structure Summary
Lattice
    abc : 5.622513960000003 16.97972225536251 28.19773534738938
 angles : 90.0 90.00000000000001 90.00000000000006
 volume : 2692.0018532808467
      A : 5.622513960000003 0.0 -9.041692047199082e-16
      B : -1.6120731005236957e-14 16.97972225536251 1.0397081255220388e-15
      C : 0.0 0.0 28.19773534738938
    pbc : True True True
PeriodicSite: Li+ (0.7028, 7.7544, 8.6481) [0.1250, 0.4567, 0.3067]
PeriodicSite: Li+ (0.7028, 2.0945, 8.6481) [0.1250, 0.1234, 0.3067]
PeriodicSite: Li+ (0.7028, 13.4143, 8.6481) [0.1250, 0.7900, 0.3067]
PeriodicSite: Li+ (3.5141, 7.7544, 8.6481) [0.6250, 0.4567, 0.3067]
PeriodicSite: Li+ (3.5141, 2.0945, 8.6481) [0.6250, 0.1234, 0.3067]
PeriodicSite: Li+ (3.5141, 13.4143, 8.6481)